In [1]:
import os

In [2]:
os.chdir('/home/tobias/Dokumente/CodingPy/Datenanalyse/PKLot_DataScience')

In [3]:
from ultralytics import YOLO
model = YOLO('yolov8n.pt')

100%|██████████| 6.25M/6.25M [00:00<00:00, 9.34MB/s]


In [4]:
results = model.train(model='yolov8n.pt', data='datasets/data.yaml', epochs=100, batch=0.5, imgsz=640, save=True, resume=False, plots=True)

New https://pypi.org/project/ultralytics/8.3.48 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.40 🚀 Python-3.12.7 torch-2.4.1.post302 CUDA:0 (NVIDIA GeForce RTX 3070, 7966MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=datasets/data.yaml, epochs=100, time=None, patience=100, batch=0.5, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, sav

RuntimeError: Dataset 'datasets/data.yaml' error ❌ '2-class dataset requires class indices 0-1, but you have invalid class indices 1-2 defined in your dataset YAML.'